In [1]:
import datetime
import statistics
import requests
from bs4 import BeautifulSoup as bs

In [2]:
right_now = datetime.datetime.now()
today = right_now.strftime("%Y-%m-%d")

In [3]:
url = 'https://api.darksky.net/forecast/'
api_key = 'XXXX'
place = {'lat': '40.811623', 'long' : '-73.962852'} #Riverside Church Harlem
options = ['?units=si']
place_url = url+api_key+"/"+place['lat']+","+place['long']+options[0]

In [4]:
place_data = requests.get(place_url).json()

In [5]:
temp = str(place_data['currently']['temperature'])
summ = str(place_data['currently']['summary'])
feels = str(place_data['daily']['data'][0]['apparentTemperatureHigh'])
hi_temp = str(place_data['daily']['data'][0]['temperatureHigh'])
lo_temp = str(place_data['daily']['data'][0]['temperatureLow'])
preciptype = str(place_data['daily']['data'][0]['precipType'])
precipprop = str(round(place_data['daily']['data'][0]['precipProbability']*100, 2))

In [6]:
todays_weather = ("Right now it is " + temp + " degrees outside and " + summ + " - Today's high will be feeling like " + feels + " degrees, with an actual maximum of " + hi_temp + " degrees and a low of " + lo_temp + " There is a " + precipprop + " percent possibility of " + preciptype + ".")

In [7]:
soup_doc_currency = bs(requests.get('https://www.xe.com/currencyconverter/convert/?Amount=1&From=EUR&To=USD').content, 'html.parser')
exchange_rate = soup_doc_currency.find(class_ = 'uccResultUnit').text

In [8]:
soup_doc_nyt = bs(requests.get('https://www.nytimes.com').content, 'html.parser')
nyt_top = soup_doc_nyt.find_all(class_ = 'story')
list_of_stories = []
for story in nyt_top:
    story_dict = {}
    try:
        headline = story.find(class_ = 'story-heading')
        byline = story.find(class_ = 'byline')
        url  = story.find('a', href=True)['href']
        story_dict['headline'] = headline.text
        if byline:
            story_dict['byline'] = byline.text
        else:
            story_dict['byline'] = '(no byline)'
        if url:
            story_dict['url'] = url
        else:    
            story_dict['url'] = 'no direct link.\nGo to https://www.nytimes.com/'
        list_of_stories.append(story_dict)
    except:
        pass
nyt_stories = str(list_of_stories[0]['headline'] + '\n' + list_of_stories[0]['byline'] + '\n' + list_of_stories[0]['url'] + '\n\n' + list_of_stories[1]['headline'] + '\n' + list_of_stories[1]['byline'] + '\n' + list_of_stories[1]['url'] + '\n\n' + list_of_stories[2]['headline'] + '\n' + list_of_stories[2]['byline'] + '\n' + list_of_stories[2]['url'])

In [9]:
soup_doc_nytger = bs(requests.get('https://www.nytimes.com/search?query=germany').content, 'html.parser')
nyt_ger_title = soup_doc_nytger.find_all('li')[1].find('h4').text
nyt_ger_shorturl = soup_doc_nytger.find_all('li')[1].find('a')['href']
nyt_ger_url = 'https://www.nytimes.com/' + nyt_ger_shorturl
nyt_ger = nyt_ger_title + '\n' + nyt_ger_url

In [10]:
soup_doc_harlem = bs(requests.get('http://experienceharlem.com/events/category/event/').content, 'html.parser')
harlem_events = soup_doc_harlem.find_all(class_ = 'vevent')
list_of_events = []
for ev in harlem_events:
    try:
        ev_dict = {}
        ev_dict['title'] = ev.find(class_ = 'entry-title').text.replace('\n', '').replace('\t', '') 
        ev_dict['date'] = ev.find(class_ = 'time-details').text 
        ev_dict['more'] = ev.find(class_ = "tribe-events-read-more")['href'] 
        list_of_events.append(ev_dict)
    except:
        pass
upcoming_events = "These events are coming up in Harlem:\n\n" + list_of_events[0]['title'] + list_of_events[0]['date'] + '\n' + list_of_events[0]['more'] + '\n\n' + list_of_events[1]['title'] + list_of_events[1]['date'] + '\n' + list_of_events[1]['more'] + '\n\n' + list_of_events[2]['title'] + list_of_events[2]['date'] + '\n' + list_of_events[2]['more'] 
#print(upcoming_events)

These events are coming up in Harlem:

AFRO FUSION CARDIO
June 5 - 6:30 pm - July 17 - 6:30 pm 
http://experienceharlem.com/event/afro-fusion-cardio-3/

HARLEM TEEN SHAKESPEARE INTENSIVE
July 9 - 3:00 am - August 17 - 6:00 pm 
http://experienceharlem.com/event/harlem-teen-shakespeare-intensive-3/

QUEENIE: Selected artworks by female artists from  El Museo del Barrio’s Collection
April 30 - 12:00 pm - June 23 - 5:00 pm 
http://experienceharlem.com/event/queenie-selected-artworks-by-female-artists-from-el-museo-del-barrios-collection/


In [11]:
email_body = todays_weather + "\n\nThe latest exchange rate is: " + exchange_rate +"\n\nHere are the top stories from the nytimes.com homepage right now:\n\n" + nyt_stories+ "\n\nAnd here is their latest story on Germany:\n\n" + nyt_ger + '\n\n' + upcoming_events

In [12]:
domain_name = 'sandboxXXX.mailgun.org'
response = requests.post("https://api.mailgun.net/v3/" + domain_name + "/messages", 
                         auth=("api", "XXX"),
                         data={"from": "WeatherBot <mailgun@sandboxXXX.mailgun.org>",
                               "to": ["XXX+mailgun@gmail.com"],
                               "subject": "Morning briefing for Jan, " + today,
                               "text": email_body}) 
response.text

'{\n  "id": "<20180620121210.1.F4B31BCA612CAAE7@sandboxXXXX.mailgun.org>",\n  "message": "Queued. Thank you."\n}'